In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

la_1718 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1718/SP1.csv")
la_1718 = la_1718[['HomeTeam','AwayTeam','FTHG','FTAG']]
la_1718 = la_1718.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
la_1718.head()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Leganes,Alaves,1,0
1,Valencia,Las Palmas,1,0
2,Celta,Sociedad,2,3
3,Girona,Ath Madrid,2,2
4,Sevilla,Espanol,1,1


In [33]:
la_1718 = la_1718[:-10]
la_1718.mean()

HomeGoals    1.545946
AwayGoals    1.143243
dtype: float64

In [34]:
# probability of draw between home and away team
skellam.pmf(0.0,  la_1718.mean()[0],  la_1718.mean()[1])

0.25271267697313393

In [35]:
# probability of home team winning by one goal
skellam.pmf(1,  la_1718.mean()[0],  la_1718.mean()[1])

0.22957890007599666

In [36]:
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([la_1718[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           la_1718[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  740
Model:                            GLM   Df Residuals:                      700
Model Family:                 Poisson   Df Model:                           39
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1033.1
Date:                Thu, 06 Sep 2018   Deviance:                       801.94
Time:                        21:05:46   Pearson chi2:                     708.
No. Iterations:                     5   Covariance Type:             nonrobust
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.1090      0.219     -0.498      0.619      -0.538       0.320
team[T.Ath Bilbao]          0.0223      0.223      0.100      0.920      -0.414       0.458
team[T.Ath Madrid]          0.3091      0.207      1.491      0.136      -0.097       0.715
team[T.Barcelona]           0.8884      0.188      4.726      0.000       0.520       1.257
team[T.Betis]               0.3775      0.206      1.834      0.067      -0.026       0.781
team[T.Celta]               0.3360      0.208      1.614      0.107      -0.072       0.744
team[T.Eibar]               0.0309      0.221      0.139      0.889      -0.403       0.464
team[T.Espanol]            -0.1463      0.232     -0.631      0.528      -0.600       0.308
team[T.Getafe]              0.0073      0.223      0.033      0.974      -0.429       0.443
team[T.Girona]              0.1974      0.214      0.921      0.357      -0.223       0.618
team[T.La Coruna]          -0.0596      0.228     -0.261      0.794      -0.507       0.388
team[T.Las Palmas]         -0.5207      0.262     -1.987      0.047      -1.034      -0.007
team[T.Leganes]            -0.2457      0.240     -1.025      0.305      -0.715       0.224
team[T.Levante]             0.0548      0.221      0.248      0.804      -0.379       0.489
team[T.Malaga]             -0.5071      0.258     -1.962      0.050      -1.014      -0.000
team[T.Real Madrid]         0.8244      0.190      4.345      0.000       0.452       1.196
team[T.Sevilla]             0.1959      0.214      0.915      0.360      -0.224       0.616
team[T.Sociedad]            0.5020      0.201      2.501      0.012       0.109       0.896
team[T.Valencia]            0.4616      0.203      2.279      0.023       0.065       0.859
team[T.Villarreal]          0.3201      0.208      1.538      0.124      -0.088       0.728
opponent[T.Ath Bilbao]     -0.0340      0.203     -0.167      0.867      -0.433       0.365
opponent[T.Ath Madrid]     -0.8918      0.266     -3.358      0.001      -1.412      -0.371
opponent[T.Barcelona]      -0.4608      0.235     -1.963      0.050      -0.921      -0.001
opponent[T.Betis]           0.1777      0.194      0.914      0.361      -0.203       0.559
opponent[T.Celta]           0.1750      0.194      0.900      0.368      -0.206       0.556
opponent[T.Eibar]          -0.0156      0.203     -0.077      0.939      -0.414       0.383
opponent[T.Espanol]        -0.1648      0.211     -0.783      0.434      -0.577       0.248
opponent[T.Getafe]         -0.4101      0.225     -1.819      0.069      -0.852       0.032
opponent[T.Girona]          0.1626      0.194      0.837      0.403      -0.218       0.544
opponent[T.La Coruna]       0.4190      0.185      2.270      0.023       0.057       0.781
opponent[T.Las Palmas]      0.3597      0.185      1.940      0.052      -0.004       0.723
opponent[T.Leganes]        -0.0128

In [37]:
poisson_model.predict(pd.DataFrame(data={'team': 'Real Madrid', 'opponent': 'Barcelona',
                                       'home':1},index=[1]))

1    1.750265
dtype: float64

In [38]:
poisson_model.predict(pd.DataFrame(data={'team': 'Barcelona', 'opponent': 'Real Madrid',
                                       'home':0},index=[1]))

1    1.986352
dtype: float64

In [39]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
simulate_match(poisson_model, 'Real Madrid', 'Barcelona', max_goals=3)

array([[0.02383462, 0.04734393, 0.04702085, 0.03113331],
       [0.04171689, 0.08286441, 0.08229893, 0.05449154],
       [0.03650779, 0.07251732, 0.07202245, 0.0476873 ],
       [0.02129943, 0.04230816, 0.04201945, 0.0278218 ]])

In [42]:
rm_bar = simulate_match(poisson_model, "Real Madrid", "Barcelona", max_goals=10)
# rm win
np.sum(np.tril(rm_bar, -1))

0.34633642333745596

In [43]:
# draw
np.sum(np.diag(rm_bar))

0.21351665721610408

In [45]:
# barcelona win
np.sum(np.triu(rm_bar, 1))

0.44013671742730287